In [1]:
!pip install langgraph langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 11.4 MB/s eta 0:00:00


In [2]:
#importing libraries

from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq


In [3]:
#defining state class
class State(TypedDict):
    query: str
    category: str
    sentiment: str
    response: str



In [4]:
#Initialize LLM

llm = ChatGroq(
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile",
    temperature=0.7
)

In [5]:
#testing the model
result = llm.invoke("who is William Samoei Ruto?")
result.content

"William Samoei Ruto is a Kenyan politician who is the current President of Kenya. He was born on December 21, 1966, in Kamagut, Uasin Gishu County, Kenya. Ruto served as the Deputy President of Kenya from 2013 to 2022 under President Uhuru Kenyatta.\n\nRuto's political career began in the 1990s, and he has held various positions, including Member of Parliament for Eldoret North Constituency and Minister for Agriculture. He was a key figure in the formation of the Jubilee Alliance, a coalition of political parties that won the 2013 general elections.\n\nIn the 2022 Kenyan general election, Ruto ran for president under the Kenya Kwanza coalition and won the election, defeating Raila Odinga, a veteran politician and former prime minister. Ruto was sworn in as the fifth President of Kenya on September 13, 2022.\n\nAs president, Ruto has pledged to focus on economic development, job creation, and improving the lives of ordinary Kenyans. He has also promised to tackle corruption and improve

In [6]:
#Helper to run prompts
def run_prompt(template: str, query: str) -> str:
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | llm
    return chain.invoke({"query": query}).content



In [7]:
#Node functions
def categorize(state: State) -> State:
    template = (
        "Categorize the following customer query into one of these categories: "
        "Technical, Billing, General. Query: {query}"
    )
    return {"category": run_prompt(template, state["query"])}

def analyze_sentiment(state: State) -> State:
    template = (
        "Analyze the sentiment of the following customer query. "
        "Respond with either 'Positive', 'Neutral', or 'Negative'. Query: {query}"
    )
    return {"sentiment": run_prompt(template, state["query"])}

def handle_technical(state: State) -> State:
    return {"response": run_prompt("Provide a technical support response to the following query: {query}", state["query"])}

def handle_billing(state: State) -> State:
    return {"response": run_prompt("Provide a billing support response to the following query: {query}", state["query"])}

def handle_general(state: State) -> State:
    return {"response": run_prompt("Provide a general support response to the following query: {query}", state["query"])}

def escalate(state: State) -> State:
    return {"response": "This query has been escalated to a human agent due to its negative sentiment."}


In [8]:
#Router

def route_query(state: State) -> str:
    if state['sentiment'] == 'Negative':
        return "escalate"
    elif state['category'] == 'Technical':
        return "handle_technical"
    elif state['category'] == 'Billing':
        return "handle_billing"
    else:
        return "handle_general"

In [11]:
# Build and compile workflow
workflow = StateGraph(State)
workflow.add_node("categorize", categorize)
workflow.add_node("analyze_sentiment", analyze_sentiment)
workflow.add_node("handle_technical", handle_technical)
workflow.add_node("handle_billing", handle_billing)
workflow.add_node("handle_general", handle_general)
workflow.add_node("escalate", escalate)

workflow.add_edge("categorize", "analyze_sentiment")
workflow.add_conditional_edges("analyze_sentiment", route_query, {
    "handle_technical": "handle_technical",
    "handle_billing": "handle_billing",
    "handle_general": "handle_general",
    "escalate": "escalate",
})
workflow.add_edge("handle_technical", END)
workflow.add_edge("handle_billing", END)
workflow.add_edge("handle_general", END)
workflow.add_edge("escalate", END)

workflow.set_entry_point("categorize")

# Compile the workflow
app = workflow.compile()


In [12]:
# Customer Support Runner
def run_customer_support(query: str) -> dict:
    result = app.invoke({"query": query})
    return {
        "category": result['category'],
        "sentiment": result['sentiment'],
        "response": result['response'],
    }


In [14]:
#testing the output
query = "I want to cancel my subscription before the next billing cycle, Can you help me?"
result = run_customer_support(query)
print(f"Query: {query}")
print(f"Category: {result['category']}")
print(f"Sentiment: {result['sentiment']}")
print(f"Response: {result['response']}")
print("\n")



Query: I want to cancel my subscription before the next billing cycle, Can you help me?
Category: The customer query can be categorized as: Billing.

The query specifically mentions canceling a subscription and the next billing cycle, which indicates a concern related to payment or subscription management, making it a billing-related issue.
Sentiment: Negative.
Response: You can cancel your subscription by following these steps:

1.  **Log in** to your account on our website or mobile app.
2.  **Go to your account settings** or profile page.
3.  **Look for the "Subscriptions" or "Membership" section**.
4.  **Click on "Cancel Subscription" or "Unsubscribe"**.
5.  **Confirm your cancellation** by clicking on the confirmation button.

If you're having trouble finding the cancellation option or need assistance with the process, our customer support team is here to help. You can contact us through our website's contact form, email, or phone number, and we'll be happy to guide you through th